In [1]:
import mysql.connector
import pandas as pd
from smtplib import SMTP
from email.mime.multipart import MIMEMultipart
from email.mime.base import MIMEBase
from email import encoders
import os

In [2]:
db = mysql.connector.connect(
    host="127.0.0.1",
    user="root",
    password="root",
    database="gdb0041"
)

In [3]:
cursor = db.cursor()
cursor.callproc('top_n_customer_by_net_sales',args=(2021,10))

(2021, 10)

In [4]:
for result in cursor.stored_results():
    data = result.fetchall()

In [5]:
df = pd.DataFrame(data, columns=[i[0] for i in cursor.description])


In [6]:
# Specify a different path where you have write permissions, such as your home directory
reports_dir = os.path.expanduser('~/Reports')
if not os.path.exists(reports_dir):
    os.makedirs(reports_dir)


In [7]:
file_path = os.path.join(reports_dir, 'report.xlsx')
df.to_excel(file_path+'report.xlsx', index=False)
try:
    df.to_excel(file_path, index=False)
    print(f"Excel file saved successfully at {file_path}")
except Exception as e:
    print(f"Failed to save Excel file: {e}")
    exit()


Excel file saved successfully at C:\Users\Sumit Rawat/Reports\report.xlsx


In [8]:
# Email credentials
email_user = 's******@gmail.com'
email_password = '**********'
email_send = 'r******@gmail.com'

subject = 'MySQL Report'

msg = MIMEMultipart()
msg['From'] = email_user
msg['To'] = email_send
msg['Subject'] = subject

try:
    with open(file_path, 'rb') as attachment:
        part = MIMEBase('application', 'octet-stream')
        part.set_payload(attachment.read())
        encoders.encode_base64(part)
        part.add_header('Content-Disposition', 'attachment; filename="report.xlsx"')
        msg.attach(part)
except Exception as e:
    print(f"Failed to read Excel file: {e}")
    exit()

try:
    server = SMTP('smtp.gmail.com', 587)
    server.starttls()
    server.login(email_user, email_password)
    server.sendmail(email_user, email_send, msg.as_string())
    server.quit()
    print("Email sent successfully.")
except Exception as e:
    print(f"Failed to send email: {e}")

Email sent successfully.
